In [3]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
from google.colab import drive
import warnings
warnings.filterwarnings("ignore")
drive.mount("/content/MyDrive/")

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


In [4]:
place_id_list = ["30","35","36","42","43","44","45","46","47","48","50","51","54","55"]
place_name_list = ["門別","盛岡","水沢","浦和","船橋","大井","川崎","金沢","笠松","名古屋","園田","姫路","高知","佐賀"]

place_id_list2 = [int(id) for id in place_id_list]

In [5]:
place_id_list = ["42","43","44","45"]
place_name_list = ["浦和","船橋","大井","川崎"]

place_id_list2 = [int(id) for id in place_id_list]

In [6]:
col_names = ["arrival_order", "frame_id", "umaban_id", "horse_name", "sex_age", "kinryo_weight",
                      "jockey", "arrival_time", "arrival_diff", "tansho", "popularity", "horse_weight", "trainer"]

In [7]:
# 2024
for place in place_id_list2:
  start = time.time()
  horse_all_df = pd.DataFrame()
  result_all_df = pd.DataFrame()
  time_all_df = pd.DataFrame()
  race_info_df = pd.DataFrame(columns = ["race_info", "race_info2"])
  for year in [2024]:
    for month in [3]:
      for day in range(1, 17):
        print(day)
        for race in range(1, 13):
          url = "https://db.netkeiba.com/race/{year}{place}{month}{day}{race}/".format(year = str(year), place = str(place), month = str(month).zfill(2), day = str(day).zfill(2), race = str(race).zfill(2))
          try:
            response = requests.get(url)
            response.encoding = "EUC-JP"
            tab = pd.read_html(response.text)

            # horse info
            horse_df = tab[0]
            horse_df.columns = col_names
            horse_df["year"] = year
            horse_df["month"] = month
            horse_df["day"] = day
            horse_df["place"] = place

            horse_df["race_id"] = race


            soup = BeautifulSoup(response.content)
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                                "a", attrs={"href": re.compile("^/horse")}
                            )
            horse_df["horse_id"] = [a_text.get('href').split("/")[-2] for a_text in horse_a_list]

            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                                "a", attrs={"href": re.compile("^/jockey")}
                            )
            horse_df["jockey_id"] = [a_text.get('href').split("/")[-2] for a_text in jockey_a_list]

            # mid order
            order_list = [re.sub('[^0-9|,]','', re.sub('-|=',',', order)) for order in pd.read_html(response.text, decimal=',', thousands='.')[4][1].values]
            for corner_i in range(0, 4):
              try:
                order = list(map(int, order_list[corner_i].split(',')))
                horse_df["mid_order_{}".format(str(corner_i + 1))] = horse_df['umaban_id'].map({horse_id: order for order, horse_id in enumerate(order, 1)})
              except:
                pass
            horse_all_df = horse_all_df.append(horse_df, ignore_index = True)

            # results
            result_df = pd.concat([tab[1], tab[2]]).reset_index(drop = True)
            result_df.columns = ["bet_type", "order", "payment", "popularity"]
            result_df["year"] = year
            result_df["month"] = month
            result_df["day"] = day
            result_df["place"] = place
            result_df["race_id"] = race
            result_all_df = result_all_df.append(result_df, ignore_index = True)


            # race info

            race_info = soup.findAll("diary_snap_cut")[0].text
            race_info2 = soup.find("p", class_ = "smalltxt").text
            race_info_df = race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)

            # mid time
            time_df = pd.read_html(response.text, decimal=',', thousands='.')[4]
            time_df.columns = ["course", "mid_order"]
            time_df["year"] = year
            time_df["month"] = month
            time_df["day"] = day
            time_df["place"] = place
            time_df["race_id"] = race
            time_all_df = time_all_df.append(time_df, ignore_index = True)


            time.sleep(0.1)
          except:
            pass
  horse_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/{}/horse_info_{}.csv".format(year, place), index = False, encoding = "utf-8")
  result_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/{}/result_{}.csv".format(year, place), index = False, encoding = "utf-8")
  race_info_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/{}/race_info_{}.csv".format(year, place), index = False, encoding = "utf-8")
  time_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/{}/mid_time_{}.csv".format(year, place), index = False, encoding = "utf-8")
  print(place)
  end = time.time()
  print((end - start) / 60)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
42
1.3717213551203409
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
43
1.8727288405100504
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
44
1.4911330858866374
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
45
1.7645662506421407


In [ ]:
horse_all_df

,arrival_order,frame_id,umaban_id,horse_name,sex_age,kinryo_weight,jockey,arrival_time,arrival_diff,tansho,...,month,day,place,race_id,horse_id,jockey_id,mid_order_1,mid_order_2,mid_order_3,mid_order_4
0,1,8,10,ダバイカンティーク,牝3,54,魚住謙心,1:32.3,NaN,5.8,...,1,5,55,1,2021100664,05604,2.0,2.0,2.0,3.0
1,2,7,8,ギャルピース,牝3,51,青海大樹,1:32.5,1,39.5,...,1,5,55,1,2021102005,05658,1.0,1.0,1.0,1.0
2,3,4,4,カーチャ,牝3,54,山下裕貴,1:32.8,1.1/2,4.3,...,1,5,55,1,2021103364,05501,5.0,5.0,5.0,2.0
3,4,7,7,ウルトラクロス,牡3,56,石川倭,1:33.3,2.1/2,1.5,...,1,5,55,1,2021106434,05534,4.0,3.0,3.0,4.0
4,5,3,3,ヌアヌノメガミ,牝3,54,金山昇馬,1:34.3,5,32.8,...,1,5,55,1,2021101887,05601,3.0,4.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,7,5,5,ボギータイサ,牡7,56,倉富隆一,1:34.2,3/4,104.7,...,1,13,55,12,2017103336,05329,5.0,5.0,6.0,8.0
620,8,8,10,ギンノアメガフル,セ5,56,加茂飛翔,1:34.4,1,35.8,...,1,13,55,12,2019102199,05633,2.0,2.0,2.0,5.0
621,9,2,2,コスモアピール,牡6,56,長田進仁,1:34.4,クビ,95.8,...,1,13,55,12,2018110148,05147,7.0,7.0,7.0,9.0
622,10,1,1,サンファボル,セ6,56,山下裕貴,1:34.9,2.1/2,15.3,...,1,13,55,12,2018106179,05501,8.0,8.0,9.0,10.0


In [ ]:
horse_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2024/sample.csv", index = False, encoding = "utf-8")

In [ ]:
for place in place_id_list2:
  start = time.time()
  horse_all_df = pd.DataFrame()
  result_all_df = pd.DataFrame()
  time_all_df = pd.DataFrame()
  race_info_df = pd.DataFrame(columns = ["race_info", "race_info2"])
  for year in range(2019, 2021):
    for month in range(1, 13):
      for day in range(1, 32):
        for race in range(1, 13):
          url = "https://db.netkeiba.com/race/{year}{place}{month}{day}{race}/".format(year = str(year), place = str(place), month = str(month).zfill(2), day = str(day).zfill(2), race = str(race).zfill(2))
          try:
            response = requests.get(url)
            response.encoding = "EUC-JP"
            tab = pd.read_html(response.text)

            # horse info
            horse_df = tab[0]
            horse_df.columns = col_names
            horse_df["year"] = year
            horse_df["month"] = month
            horse_df["day"] = day
            horse_df["place"] = place

            horse_df["race_id"] = race


            soup = BeautifulSoup(response.content)
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                                "a", attrs={"href": re.compile("^/horse")}
                            )
            horse_df["horse_id"] = [a_text.get('href').split("/")[-2] for a_text in horse_a_list]

            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                                "a", attrs={"href": re.compile("^/jockey")}
                            )
            horse_df["jockey_id"] = [a_text.get('href').split("/")[-2] for a_text in jockey_a_list]

            # mid order
            order_list = [re.sub('[^0-9|,]','', re.sub('-|=',',', order)) for order in pd.read_html(response.text, decimal=',', thousands='.')[4][1].values]
            for corner_i in range(0, 4):
              try:
                order = list(map(int, order_list[corner_i].split(',')))
                horse_df["mid_order_{}".format(str(corner_i + 1))] = horse_df['umaban_id'].map({horse_id: order for order, horse_id in enumerate(order, 1)})
              except:
                pass
            horse_all_df = horse_all_df.append(horse_df, ignore_index = True)

            # results
            result_df = pd.concat([tab[1], tab[2]]).reset_index(drop = True)
            result_df.columns = ["bet_type", "order", "payment", "popularity"]
            result_df["year"] = year
            result_df["month"] = month
            result_df["day"] = day
            result_df["place"] = place
            result_df["race_id"] = race
            result_all_df = result_all_df.append(result_df, ignore_index = True)


            # race info

            race_info = soup.findAll("diary_snap_cut")[0].text
            race_info2 = soup.find("p", class_ = "smalltxt").text
            race_info_df = race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)

            # mid time
            time_df = pd.read_html(response.text, decimal=',', thousands='.')[4]
            time_df.columns = ["course", "mid_order"]
            time_df["year"] = year
            time_df["month"] = month
            time_df["day"] = day
            time_df["place"] = place
            time_df["race_id"] = race
            time_all_df = time_all_df.append(time_df, ignore_index = True)


            time.sleep(0.5)
          except:
            pass
  horse_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/{}/horse_info_{}.csv".format(year, place), index = False, encoding = "utf-8")
  result_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/{}/result_{}.csv".format(year, place), index = False, encoding = "utf-8")
  race_info_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/{}/race_info_{}.csv".format(year, place), index = False, encoding = "utf-8")
  time_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/{}/mid_time_{}.csv".format(year, place), index = False, encoding = "utf-8")
  print(place)
  end = time.time()
  print((end - start) / 60)

30
95.76065518856049
35
89.67455486456554
36
86.82892145315806
42
86.9371177872022
43
88.81071515083313
44
104.90172502994537
45
89.82949231863022


In [ ]:
print(time.time())
for place in place_id_list2[8:]:
  horse_all_df = pd.DataFrame()
  result_all_df = pd.DataFrame()
  time_all_df = pd.DataFrame()
  race_info_df = pd.DataFrame(columns = ["race_info", "race_info2"])
  for year in [2023]:
    for month in range(1, 13):
      for day in range(1, 32):
        for race in range(1, 13):
          url = "https://db.netkeiba.com/race/{year}{place}{month}{day}{race}/".format(year = str(year), place = str(place), month = str(month).zfill(2), day = str(day).zfill(2), race = str(race).zfill(2))
          try:
            response = requests.get(url)
            response.encoding = "EUC-JP"
            tab = pd.read_html(response.text)

            # horse info
            horse_df = tab[0]
            horse_df.columns = col_names
            horse_df["year"] = year
            horse_df["month"] = month
            horse_df["day"] = day
            horse_df["place"] = place

            horse_df["race_id"] = race


            soup = BeautifulSoup(response.content)
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                                "a", attrs={"href": re.compile("^/horse")}
                            )
            horse_df["horse_id"] = [a_text.get('href').split("/")[-2] for a_text in horse_a_list]

            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                                "a", attrs={"href": re.compile("^/jockey")}
                            )
            horse_df["jockey_id"] = [a_text.get('href').split("/")[-2] for a_text in jockey_a_list]

            # mid order
            order_list = [re.sub('[^0-9|,]','', re.sub('-|=',',', order)) for order in pd.read_html(response.text, decimal=',', thousands='.')[4][1].values]
            for corner_i in range(0, 4):
              order = list(map(int, order_list[corner_i].split(',')))
              horse_df["mid_order_{}".format(str(corner_i + 1))] = horse_df['umaban_id'].map({horse_id: order for order, horse_id in enumerate(order, 1)})
            horse_all_df = horse_all_df.append(horse_df, ignore_index = True)

            # results
            result_df = pd.concat([tab[1], tab[2]]).reset_index(drop = True)
            result_df.columns = ["bet_type", "order", "payment", "popularity"]
            result_df["year"] = year
            result_df["month"] = month
            result_df["day"] = day
            result_df["place"] = place
            result_df["race_id"] = race
            result_all_df = result_all_df.append(result_df, ignore_index = True)


            # race info

            race_info = soup.findAll("diary_snap_cut")[0].text
            race_info2 = soup.find("p", class_ = "smalltxt").text
            race_info_df = race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)

            # mid time
            time_df = pd.read_html(response.text, decimal=',', thousands='.')[4]
            time_df.columns = ["course", "mid_order"]
            time_df["year"] = year
            time_df["month"] = month
            time_df["day"] = day
            time_df["place"] = place
            time_df["race_id"] = race
            time_all_df = time_all_df.append(time_df, ignore_index = True)


            time.sleep(0.5)
          except:
            pass
  horse_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2022/horse_info_{}.csv".format(place), index = False, encoding = "utf-8")
  result_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2022/result_{}.csv".format(place), index = False, encoding = "utf-8")
  race_info_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2022/race_info_{}.csv".format(place), index = False, encoding = "utf-8")
  time_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2022/mid_time_{}.csv".format(place), index = False, encoding = "utf-8")
  print(place)
  print(time.time())

In [ ]:
print(time.time())
for place in place_id_list2[-1:]:
  horse_all_df = pd.DataFrame()
  result_all_df = pd.DataFrame()
  time_all_df = pd.DataFrame()
  race_info_df = pd.DataFrame(columns = ["race_info", "race_info2"])
  for year in [2021]:
    for month in range(1, 13):
      print("{}_{}".format(place, month))
      for day in range(1, 32):
        for race in range(1, 13):
          url = "https://db.netkeiba.com/race/{year}{place}{month}{day}{race}/".format(year = str(year), place = str(place), month = str(month).zfill(2), day = str(day).zfill(2), race = str(race).zfill(2))
          try:
            response = requests.get(url)
            response.encoding = "EUC-JP"
            tab = pd.read_html(response.text)

            # horse info
            horse_df = tab[0]
            horse_df.columns = col_names
            horse_df["year"] = year
            horse_df["month"] = month
            horse_df["day"] = day
            horse_df["place"] = place

            horse_df["race_id"] = race


            soup = BeautifulSoup(response.content)
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                                "a", attrs={"href": re.compile("^/horse")}
                            )
            horse_df["horse_id"] = [a_text.get('href').split("/")[-2] for a_text in horse_a_list]

            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                                "a", attrs={"href": re.compile("^/jockey")}
                            )
            horse_df["jockey_id"] = [a_text.get('href').split("/")[-2] for a_text in jockey_a_list]

            # mid order
            order_list = [re.sub('[^0-9|,]','', re.sub('-|=',',', order)) for order in pd.read_html(response.text, decimal=',', thousands='.')[4][1].values]
            for corner_i in range(0, 4):
              order = list(map(int, order_list[corner_i].split(',')))
              horse_df["mid_order_{}".format(str(corner_i + 1))] = horse_df['umaban_id'].map({horse_id: order for order, horse_id in enumerate(order, 1)})
            horse_all_df = horse_all_df.append(horse_df, ignore_index = True)

            # results
            result_df = pd.concat([tab[1], tab[2]]).reset_index(drop = True)
            result_df.columns = ["bet_type", "order", "payment", "popularity"]
            result_df["year"] = year
            result_df["month"] = month
            result_df["day"] = day
            result_df["place"] = place
            result_df["race_id"] = race
            result_all_df = result_all_df.append(result_df, ignore_index = True)


            # race info

            race_info = soup.findAll("diary_snap_cut")[0].text
            race_info2 = soup.find("p", class_ = "smalltxt").text
            race_info_df = race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)

            # mid time
            time_df = pd.read_html(response.text, decimal=',', thousands='.')[4]
            time_df.columns = ["course", "mid_order"]
            time_df["year"] = year
            time_df["month"] = month
            time_df["day"] = day
            time_df["place"] = place
            time_df["race_id"] = race
            time_all_df = time_all_df.append(time_df, ignore_index = True)


            time.sleep(0.5)
          except:
            pass
  horse_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2021/horse_info_{}.csv".format(place), index = False, encoding = "utf-8")
  result_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2021/result_{}.csv".format(place), index = False, encoding = "utf-8")
  race_info_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2021/race_info_{}.csv".format(place), index = False, encoding = "utf-8")
  time_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2021/mid_time_{}.csv".format(place), index = False, encoding = "utf-8")
  print(place)
  print(time.time())

1705315742.4905756
55_1
55_2
55_3
55_4
55_5
55_6
55_7
55_8
55_9
55_10
55_11
55_12
55
1705318768.2295344


In [ ]:
print(time.time())
for place in place_id_list2[-3:]:
  horse_all_df = pd.DataFrame()
  result_all_df = pd.DataFrame()
  time_all_df = pd.DataFrame()
  race_info_df = pd.DataFrame(columns = ["race_info", "race_info2"])
  for year in [2022]:
    for month in range(1, 13):
      print("{}_{}".format(place, month))
      for day in range(1, 32):
        for race in range(1, 13):
          url = "https://db.netkeiba.com/race/{year}{place}{month}{day}{race}/".format(year = str(year), place = str(place), month = str(month).zfill(2), day = str(day).zfill(2), race = str(race).zfill(2))
          try:
            response = requests.get(url)
            response.encoding = "EUC-JP"
            tab = pd.read_html(response.text)

            # horse info
            horse_df = tab[0]
            horse_df.columns = col_names
            horse_df["year"] = year
            horse_df["month"] = month
            horse_df["day"] = day
            horse_df["place"] = place

            horse_df["race_id"] = race


            soup = BeautifulSoup(response.content)
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                                "a", attrs={"href": re.compile("^/horse")}
                            )
            horse_df["horse_id"] = [a_text.get('href').split("/")[-2] for a_text in horse_a_list]

            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                                "a", attrs={"href": re.compile("^/jockey")}
                            )
            horse_df["jockey_id"] = [a_text.get('href').split("/")[-2] for a_text in jockey_a_list]

            # mid order
            order_list = [re.sub('[^0-9|,]','', re.sub('-|=',',', order)) for order in pd.read_html(response.text, decimal=',', thousands='.')[4][1].values]
            for corner_i in range(0, 4):
              order = list(map(int, order_list[corner_i].split(',')))
              horse_df["mid_order_{}".format(str(corner_i + 1))] = horse_df['umaban_id'].map({horse_id: order for order, horse_id in enumerate(order, 1)})
            horse_all_df = horse_all_df.append(horse_df, ignore_index = True)

            # results
            result_df = pd.concat([tab[1], tab[2]]).reset_index(drop = True)
            result_df.columns = ["bet_type", "order", "payment", "popularity"]
            result_df["year"] = year
            result_df["month"] = month
            result_df["day"] = day
            result_df["place"] = place
            result_df["race_id"] = race
            result_all_df = result_all_df.append(result_df, ignore_index = True)


            # race info

            race_info = soup.findAll("diary_snap_cut")[0].text
            race_info2 = soup.find("p", class_ = "smalltxt").text
            race_info_df = race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)

            # mid time
            time_df = pd.read_html(response.text, decimal=',', thousands='.')[4]
            time_df.columns = ["course", "mid_order"]
            time_df["year"] = year
            time_df["month"] = month
            time_df["day"] = day
            time_df["place"] = place
            time_df["race_id"] = race
            time_all_df = time_all_df.append(time_df, ignore_index = True)


            time.sleep(0.5)
          except:
            pass
  horse_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2022/horse_info_{}.csv".format(place), index = False, encoding = "utf-8")
  result_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2022/result_{}.csv".format(place), index = False, encoding = "utf-8")
  race_info_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2022/race_info_{}.csv".format(place), index = False, encoding = "utf-8")
  time_all_df.to_csv("/content/MyDrive/MyDrive/horse/data/src/2022/mid_time_{}.csv".format(place), index = False, encoding = "utf-8")
  print(place)
  print(time.time())

1705318768.2712018
51_1
51_2
51_3
51_4
51_5
51_6
51_7
51_8
51_9
51_10
51_11
51_12
51
1705320815.6812634
54_1
54_2
54_3
54_4
54_5
54_6
54_7
54_8
54_9
54_10
54_11
54_12
54
1705323751.2672048
55_1
55_2
55_3
55_4
55_5
55_6
55_7
55_8
55_9
55_10
55_11
55_12
55
1705326778.7025716


In [ ]:
!apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (143 kB/s)
Selecting previously unselected package tree.
(Reading database ... 121654 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!tree "/content/MyDrive/MyDrive/horse/data/src/"

/content/MyDrive/MyDrive/horse/data/src/
├── 2021
│   ├── horse_info_30.csv
│   ├── horse_info_35.csv
│   ├── horse_info_36.csv
│   ├── horse_info_42.csv
│   ├── horse_info_43.csv
│   ├── horse_info_44.csv
│   ├── horse_info_45.csv
│   ├── horse_info_46.csv
│   ├── horse_info_47.csv
│   ├── horse_info_48.csv
│   ├── horse_info_50.csv
│   ├── horse_info_51.csv
│   ├── horse_info_54.csv
│   ├── horse_info_55.csv
│   ├── mid_time_30.csv
│   ├── mid_time_35.csv
│   ├── mid_time_36.csv
│   ├── mid_time_42.csv
│   ├── mid_time_43.csv
│   ├── mid_time_44.csv
│   ├── mid_time_45.csv
│   ├── mid_time_46.csv
│   ├── mid_time_47.csv
│   ├── mid_time_48.csv
│   ├── mid_time_50.csv
│   ├── mid_time_51.csv
│   ├── mid_time_54.csv
│   ├── mid_time_55.csv
│   ├── race_info_30.csv
│   ├── race_info_35.csv
│   ├── race_info_36.csv
│   ├── race_info_42.csv
│   ├── race_info_43.csv
│   ├── race_info_44.csv
│   ├── race_info_45.csv
│   ├── race_info_46.csv
│   ├── race_info_47.csv
│   ├── race_info_48.csv


In [ ]:
a = pd.read_html("https://db.netkeiba.com/race/202347101210/")

In [ ]:
response = requests.get("https://db.netkeiba.com/race/202347101210/")
response.encoding = "EUC-JP"
soup = BeautifulSoup(response.content)

False

In [ ]:
print(time.time())
for place in [47]:
  horse_all_df = pd.DataFrame()
  result_all_df = pd.DataFrame()
  time_all_df = pd.DataFrame()
  race_info_df = pd.DataFrame(columns = ["race_info", "race_info2"])
  for year in [2023]:
    for month in [10]:
      print("{}_{}".format(place, month))
      for day in [12]:
        for race in [10]:
          url = "https://db.netkeiba.com/race/{year}{place}{month}{day}{race}/".format(year = str(year), place = str(place), month = str(month).zfill(2), day = str(day).zfill(2), race = str(race).zfill(2))

          response = requests.get(url)
          response.encoding = "EUC-JP"
          tab = pd.read_html(response.text)
          try:
          # horse info
          horse_df = tab[0]
          horse_df.columns = col_names
          horse_df["year"] = year
          horse_df["month"] = month
          horse_df["day"] = day
          horse_df["place"] = place

          horse_df["race_id"] = race


          soup = BeautifulSoup(response.content)
          horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                              "a", attrs={"href": re.compile("^/horse")}
                          )
          horse_df["horse_id"] = [a_text.get('href').split("/")[-2] for a_text in horse_a_list]

          jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                              "a", attrs={"href": re.compile("^/jockey")}
                          )
          horse_df["jockey_id"] = [a_text.get('href').split("/")[-2] for a_text in jockey_a_list]

          # mid order
          order_list = [re.sub('[^0-9|,]','', re.sub('-|=',',', order)) for order in pd.read_html(response.text, decimal=',', thousands='.')[4][1].values]
          for corner_i in range(0, 4):
            try:
              order = list(map(int, order_list[corner_i].split(',')))
              horse_df["mid_order_{}".format(str(corner_i + 1))] = horse_df['umaban_id'].map({horse_id: order for order, horse_id in enumerate(order, 1)})
            except:
              pass
          horse_all_df = horse_all_df.append(horse_df, ignore_index = True)

          # results
          result_df = pd.concat([tab[1], tab[2]]).reset_index(drop = True)
          result_df.columns = ["bet_type", "order", "payment", "popularity"]
          result_df["year"] = year
          result_df["month"] = month
          result_df["day"] = day
          result_df["place"] = place
          result_df["race_id"] = race
          result_all_df = result_all_df.append(result_df, ignore_index = True)


          # race info

          race_info = soup.findAll("diary_snap_cut")[0].text
          race_info2 = soup.find("p", class_ = "smalltxt").text
          race_info_df = race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)

          # mid time
          time_df = pd.read_html(response.text, decimal=',', thousands='.')[4]
          time_df.columns = ["course", "mid_order"]
          time_df["year"] = year
          time_df["month"] = month
          time_df["day"] = day
          time_df["place"] = place
          time_df["race_id"] = race
          time_all_df = time_all_df.append(time_df, ignore_index = True)


          time.sleep(0.5)
  print(place)
  print(time.time())

1705327808.218505
47_10
47
1705327809.204093


In [ ]:
horse_all_df

,arrival_order,frame_id,umaban_id,horse_name,sex_age,kinryo_weight,jockey,arrival_time,arrival_diff,tansho,...,month,day,place,race_id,horse_id,jockey_id,mid_order_1,mid_order_2,mid_order_3,mid_order_4
0,1,4,4,ブルグミューラー,牡3,56,笹川翼,1:32.4,NaN,1.1,...,1,10,42,1,2020101927,05524,1,1,1,1
1,2,7,8,リバースイート,牝3,54,山本聡紀,1:32.6,1,7.6,...,1,10,42,1,2020101972,a02b0,3,3,2,2
2,3,8,9,スマイルマハロ,牡3,56,秋元耕成,1:34.2,8,41.1,...,1,10,42,1,2020105898,05347,4,6,4,3
3,4,5,5,リオチャン,牡3,56,達城龍次,1:34.3,1/2,28.5,...,1,10,42,1,2020101976,05540,9,10,9,8
4,5,3,3,ワールドバトル,牡3,56,町田直希,1:34.5,1,39.8,...,1,10,42,1,2020104602,05455,6,4,6,5
5,6,7,7,アイゴールド,牝3,53,新原周馬,1:35.2,3,36.3,...,1,10,42,1,2020100319,05647,2,2,3,4
6,7,6,6,エマーブルクラス,牝3,54,高橋哲也,1:35.4,1,78.7,...,1,10,42,1,2020101440,a017e,10,9,5,6
7,8,2,2,シナノフローレンス,牝3,53,七夕裕次,1:35.8,2,23.0,...,1,10,42,1,2020103162,05611,5,5,7,7
8,9,1,1,プリンスオーソ,牡3,56,加藤和博,1:36.5,3,82.7,...,1,10,42,1,2020104217,05248,8,7,8,9
9,10,8,10,アイトアーサー,牡3,56,福原杏,1:41.2,大,10.6,...,1,10,42,1,2020102265,05595,7,8,10,10


In [ ]:
horse_df

,arrival_order,frame_id,umaban_id,horse_name,sex_age,kinryo_weight,jockey,arrival_time,arrival_diff,tansho,popularity,horse_weight,trainer
0,1,5,5,ラブリーギャル,牝7,54,見越彬央,1:32.2,NaN,3.5,1.0,503(+2),[地] 海馬澤司
1,2,2,2,ハーモニージョーイ,牡6,56,笹川翼,1:32.2,クビ,7.2,5.0,470(-6),[地] 新井清重
2,3,6,8,ビアンカグリーン,牡8,56,森泰斗,1:32.5,1.1/2,4.9,3.0,487(-3),[地] 酒井一則
3,4,7,9,ラウズアップ,牡7,56,藤江渉,1:33.6,5,9.6,6.0,429(-5),[地] 牛房栄吉
4,5,6,7,オールドボンバー,牡7,56,加藤和博,1:34.0,2,7.1,4.0,485(-2),[地] 横山保
5,6,7,10,ナツィオナーレ,牝8,50,中島良美,1:34.1,3/4,22.0,8.0,461(+20),[地] 野口孝
6,7,1,1,スマイルジーニアス,セ8,56,福原杏,1:34.5,2,4.4,2.0,513(0),[地] 鹿沼良和
7,8,3,3,アミン,牝6,54,岡田大,1:34.6,3/4,54.7,10.0,474(+1),[地] 冨田敏男
8,9,8,11,ラッキーアモン,牡7,56,寺島憂人,1:35.2,3,36.8,9.0,522(+10),[地] 箕輪武
9,10,8,12,ゴールドスペンサー,牡5,56,秋元耕成,1:35.2,クビ,161.5,11.0,482(-10),[地] 吉田正美


In [ ]:
# horse info
horse_df = tab[0]
horse_df.columns = col_names
horse_df["year"] = 2023
horse_df["month"] = 1
horse_df["day"] = 2
horse_df["place"] = 3

horse_df["race_id"] = 4


soup = BeautifulSoup(response.content)
horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/horse")}
                )
horse_df["horse_id"] = [a_text.get('href').split("/")[-2] for a_text in horse_a_list]

jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
horse_df["jockey_id"] = [a_text.get('href').split("/")[-2] for a_text in jockey_a_list]

# mid order
order_list = [re.sub('[^0-9|,]','', re.sub('-|=',',', order)) for order in pd.read_html(response.text, decimal=',', thousands='.')[4][1].values]
for corner_i in range(0, 4):
  order = list(map(int, order_list[corner_i].split(',')))
  horse_df["mid_order_{}".format(str(corner_i + 1))] = horse_df['umaban_id'].map({horse_id: order for order, horse_id in enumerate(order, 1)})

# results
result_df = pd.concat([tab[1], tab[2]]).reset_index(drop = True)
result_df.columns = ["bet_type", "order", "payment", "popularity"]
result_df["year"] = year
result_df["month"] = month
result_df["day"] = day
result_df["place"] = place
result_df["race_id"] = race


# race info

race_info = soup.findAll("diary_snap_cut")[0].text
race_info2 = soup.find("p", class_ = "smalltxt").text
race_info_df = race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)

# mid time
time_df = pd.read_html(response.text, decimal=',', thousands='.')[4]
time_df.columns = ["course", "mid_order"]
time_df["year"] = year
time_df["month"] = month
time_df["day"] = day
time_df["place"] = place
time_df["race_id"] = race

time.sleep(0.5)
print(url)

ValueError: Length mismatch: Expected axis has 24 elements, new values have 13 elements

In [ ]:
race_info = soup.findAll("diary_snap_cut")[0].text
race_info2 = soup.find("p", class_ = "smalltxt").text
race_info_df = race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)

<ipython-input-74-59b512fd0ef5>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)


,race_info,race_info2
0,\nダ左1000m / 天候 : 晴 / ダート : 良 / 発走 : 20:50\n\n過...,2023年10月25日 7回船橋3日目


In [ ]:
race_info2

'\nﾀｲﾑ指数\n通過\n上り\n'

# Family Network

In [ ]:
https://db.netkeiba.com/horse/2020105491/

'https://db.netkeiba.com/race/202442011004/'

In [ ]:
start = time.time()
year = 2024
place = 42
month = 1
day = 10
race = 4

horse_all_df = pd.DataFrame()
result_all_df = pd.DataFrame()
race_info_df = pd.DataFrame(columns = ["race_info", "race_info2"])
arrival_df = pd.DataFrame(columns = ["corner", "arrival_order"])
url = "https://db.netkeiba.com/race/{year}{place}{month}{day}{race}/".format(year = str(year), place = str(place), month = str(month).zfill(2), day = str(day).zfill(2), race = str(race).zfill(2))

response = requests.get(url)
response.encoding = "EUC-JP"
tab = pd.read_html(response.text)

# horse info
horse_df = tab[0]
horse_df.columns = col_names
horse_df["year"] = year
horse_df["month"] = month
horse_df["day"] = day
horse_df["place"] = place

horse_df["race_id"] = race


soup = BeautifulSoup(response.content)
horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/horse")}
                )
horse_df["horse_id"] = [a_text.get('href').split("/")[-2] for a_text in horse_a_list]

jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
horse_df["jockey_id"] = [a_text.get('href').split("/")[-2] for a_text in jockey_a_list]

# mid order
order_list = [re.sub('[^0-9|,]','', re.sub('-|=',',', order)) for order in pd.read_html(response.text, decimal=',', thousands='.')[4][1].values]
for corner_i in range(0, 4):
  order = list(map(int, order_list[corner_i].split(',')))
  horse_df["mid_order_{}".format(str(corner_i + 1))] = horse_df['umaban_id'].map({horse_id: order for order, horse_id in enumerate(order, 1)})
horse_all_df = horse_all_df.append(horse_df, ignore_index = True)

# results
result_df = pd.concat([tab[1], tab[2]]).reset_index(drop = True)
result_df.columns = ["bet_type", "order", "payment", "popularity"]
result_df["year"] = year
result_df["month"] = month
result_df["day"] = day
result_df["place"] = place
result_df["race_id"] = race
result_all_df = result_all_df.append(result_df, ignore_index = True)


# race info

race_info = soup.findAll("diary_snap_cut")[0].text
race_info2 = soup.find("p", class_ = "smalltxt").text
race_info_df = race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)



time_df = pd.read_html(response.text, decimal=',', thousands='.')[4]
time_df.columns = ["course", "mid_order"]
time_df["year"] = year
time_df["month"] = month
time_df["day"] = day
time_df["place"] = place
time_df["race_id"] = race
time_all_df = time_all_df.append(time_df, ignore_index = True)


time.sleep(0.5)
end = time.time()

In [ ]:
end - start

1.336364984512329

In [ ]:
1.336364984512329 * 14000 / 60

311.81849638621014

In [ ]:
horse_df

,arrival_order,frame_id,umaban_id,horse_name,sex_age,kinryo_weight,jockey,arrival_time,arrival_diff,tansho,...,day,place,race_id,horse_id,jockey_id,order,mid_order_1,mid_order_2,mid_order_3,mid_order_4
0,1,8,11,ニシノスイセイ,牡4,56,福原杏,1:32.3,NaN,4.3,...,10,42,4,2020105491,05595,2.0,2.0,2.0,2.0,2.0
1,2,8,12,リュウノファントム,牡4,56,桑村真明,1:32.5,3/4,7.3,...,10,42,4,2020101125,05411,1.0,1.0,1.0,1.0,1.0
2,3,3,3,ミーティアカフェ,牝6,53,七夕裕次,1:32.6,3/4,5.0,...,10,42,4,2018106206,05611,9.0,9.0,10.0,10.0,6.0
3,4,7,10,メイショウポルポ,牡7,56,和田譲治,1:32.6,アタマ,3.9,...,10,42,4,2017104517,05399,6.0,6.0,5.0,5.0,5.0
4,5,7,9,サトノトリガー,牡7,56,吉留孝司,1:33.0,2,23.7,...,10,42,4,2017100379,05397,10.0,10.0,8.0,7.0,7.0
5,6,6,8,ヤマタケコーチャン,牡5,56,高橋哲也,1:33.1,3/4,8.3,...,10,42,4,2019102153,a017e,4.0,4.0,4.0,3.0,4.0
6,7,5,6,アナザーヴィータ,牡6,56,見越彬央,1:33.2,1/2,39.7,...,10,42,4,2018101623,a02c4,3.0,3.0,3.0,4.0,3.0
7,8,1,1,ソウヤミサキ,牝6,54,落合玄太,1:33.5,1.1/2,5.2,...,10,42,4,2018106732,05578,8.0,8.0,9.0,9.0,9.0
8,9,2,2,クラマサヴィーナス,牝4,53,木間塚龍,1:34.0,2.1/2,110.6,...,10,42,4,2020101437,05607,5.0,5.0,6.0,6.0,8.0
9,10,5,5,キョウエイルーマー,牡4,56,藤本現暉,1:35.5,7,21.3,...,10,42,4,2020109074,05590,11.0,11.0,11.0,11.0,11.0


In [ ]:
order_ls

In [ ]:
order

[12, 11, 6, 8, 2, 10, 7, 1, 3, 9, 5]

In [ ]:
horse_df

,arrival_order,frame_id,umaban_id,horse_name,sex_age,kinryo_weight,jockey,arrival_time,arrival_diff,tansho,...,horse_weight,trainer,year,month,day,place,race_id,horse_id,jockey_id,order
0,1,8,11,ニシノスイセイ,牡4,56,福原杏,1:32.3,NaN,4.3,...,456(-13),[地] 小澤宏次,2024,1,10,42,4,2020105491,05595,2.0
1,2,8,12,リュウノファントム,牡4,56,桑村真明,1:32.5,3/4,7.3,...,475(+2),[地] 野口孝,2024,1,10,42,4,2020101125,05411,1.0
2,3,3,3,ミーティアカフェ,牝6,53,七夕裕次,1:32.6,3/4,5.0,...,475(-3),[地] 箕輪武,2024,1,10,42,4,2018106206,05611,9.0
3,4,7,10,メイショウポルポ,牡7,56,和田譲治,1:32.6,アタマ,3.9,...,451(+5),[地] 宇野木博,2024,1,10,42,4,2017104517,05399,6.0
4,5,7,9,サトノトリガー,牡7,56,吉留孝司,1:33.0,2,23.7,...,470(+4),[地] 小林真治,2024,1,10,42,4,2017100379,05397,10.0
5,6,6,8,ヤマタケコーチャン,牡5,56,高橋哲也,1:33.1,3/4,8.3,...,442(+3),[地] 水野貴史,2024,1,10,42,4,2019102153,a017e,4.0
6,7,5,6,アナザーヴィータ,牡6,56,見越彬央,1:33.2,1/2,39.7,...,496(-2),[地] 鈴木勝文,2024,1,10,42,4,2018101623,a02c4,3.0
7,8,1,1,ソウヤミサキ,牝6,54,落合玄太,1:33.5,1.1/2,5.2,...,454(0),[地] 平山真希,2024,1,10,42,4,2018106732,05578,8.0
8,9,2,2,クラマサヴィーナス,牝4,53,木間塚龍,1:34.0,2.1/2,110.6,...,472(-5),[地] 矢野義幸,2024,1,10,42,4,2020101437,05607,5.0
9,10,5,5,キョウエイルーマー,牡4,56,藤本現暉,1:35.5,7,21.3,...,486(+17),[地] 小澤宏次,2024,1,10,42,4,2020109074,05590,11.0


In [ ]:
order.split(",")

['12', '11', '6', '8', '10', '2', '7', '9', '1', '3', '5']

In [ ]:
order_list[3].split(",")

['12', '11', '6', '8', '10', '3', '9', '2', '1', '7', '5']

In [ ]:
horse_df

,arrival_order,frame_id,umaban_id,horse_name,sex_age,kinryo_weight,jockey,arrival_time,arrival_diff,tansho,popularity,horse_weight,trainer,year,month,day,place,race_id,horse_id,jockey_id
0,1,8,11,ニシノスイセイ,牡4,56,福原杏,1:32.3,NaN,4.3,2.0,456(-13),[地] 小澤宏次,2024,1,10,42,4,2020105491,05595
1,2,8,12,リュウノファントム,牡4,56,桑村真明,1:32.5,3/4,7.3,5.0,475(+2),[地] 野口孝,2024,1,10,42,4,2020101125,05411
2,3,3,3,ミーティアカフェ,牝6,53,七夕裕次,1:32.6,3/4,5.0,3.0,475(-3),[地] 箕輪武,2024,1,10,42,4,2018106206,05611
3,4,7,10,メイショウポルポ,牡7,56,和田譲治,1:32.6,アタマ,3.9,1.0,451(+5),[地] 宇野木博,2024,1,10,42,4,2017104517,05399
4,5,7,9,サトノトリガー,牡7,56,吉留孝司,1:33.0,2,23.7,8.0,470(+4),[地] 小林真治,2024,1,10,42,4,2017100379,05397
5,6,6,8,ヤマタケコーチャン,牡5,56,高橋哲也,1:33.1,3/4,8.3,6.0,442(+3),[地] 水野貴史,2024,1,10,42,4,2019102153,a017e
6,7,5,6,アナザーヴィータ,牡6,56,見越彬央,1:33.2,1/2,39.7,9.0,496(-2),[地] 鈴木勝文,2024,1,10,42,4,2018101623,a02c4
7,8,1,1,ソウヤミサキ,牝6,54,落合玄太,1:33.5,1.1/2,5.2,4.0,454(0),[地] 平山真希,2024,1,10,42,4,2018106732,05578
8,9,2,2,クラマサヴィーナス,牝4,53,木間塚龍,1:34.0,2.1/2,110.6,10.0,472(-5),[地] 矢野義幸,2024,1,10,42,4,2020101437,05607
9,10,5,5,キョウエイルーマー,牡4,56,藤本現暉,1:35.5,7,21.3,7.0,486(+17),[地] 小澤宏次,2024,1,10,42,4,2020109074,05590


In [ ]:
horse_df["horse_id"] = horse_id_list
horse_df["jockey_id"] = jockey_id_list

In [ ]:
horse_df

,arrival_order,frame_id,horse_id,horse_name,sex_age,kinryo_weight,jockey,arrival_time,arrival_diff,tansho,popularity,horse_weight,trainer,year,month,day,place,race_id,jockey_id
0,1,8,2020105491,ニシノスイセイ,牡4,56,福原杏,1:32.3,NaN,4.3,2.0,456(-13),[地] 小澤宏次,2024,1,10,42,4,05595
1,2,8,2020101125,リュウノファントム,牡4,56,桑村真明,1:32.5,3/4,7.3,5.0,475(+2),[地] 野口孝,2024,1,10,42,4,05411
2,3,3,2018106206,ミーティアカフェ,牝6,53,七夕裕次,1:32.6,3/4,5.0,3.0,475(-3),[地] 箕輪武,2024,1,10,42,4,05611
3,4,7,2017104517,メイショウポルポ,牡7,56,和田譲治,1:32.6,アタマ,3.9,1.0,451(+5),[地] 宇野木博,2024,1,10,42,4,05399
4,5,7,2017100379,サトノトリガー,牡7,56,吉留孝司,1:33.0,2,23.7,8.0,470(+4),[地] 小林真治,2024,1,10,42,4,05397
5,6,6,2019102153,ヤマタケコーチャン,牡5,56,高橋哲也,1:33.1,3/4,8.3,6.0,442(+3),[地] 水野貴史,2024,1,10,42,4,a017e
6,7,5,2018101623,アナザーヴィータ,牡6,56,見越彬央,1:33.2,1/2,39.7,9.0,496(-2),[地] 鈴木勝文,2024,1,10,42,4,a02c4
7,8,1,2018106732,ソウヤミサキ,牝6,54,落合玄太,1:33.5,1.1/2,5.2,4.0,454(0),[地] 平山真希,2024,1,10,42,4,05578
8,9,2,2020101437,クラマサヴィーナス,牝4,53,木間塚龍,1:34.0,2.1/2,110.6,10.0,472(-5),[地] 矢野義幸,2024,1,10,42,4,05607
9,10,5,2020109074,キョウエイルーマー,牡4,56,藤本現暉,1:35.5,7,21.3,7.0,486(+17),[地] 小澤宏次,2024,1,10,42,4,05590


In [ ]:
url = "https://db.netkeiba.com/race/201901010101"

year = 2019

response = requests.get(url)
tab = pd.read_html(response.text)
# horse info
horse_df = tab[0]
horse_df.columns = col_names
horse_df["year"] = year
horse_df["month"] = month
horse_df["day"] = day
horse_df["place"] = place

horse_df["race_id"] = race
horse_all_df = horse_all_df.append(horse_df, ignore_index = True)

# results
result_df = pd.concat([tab[1], tab[2]]).reset_index(drop = True)
result_df.columns = ["bet_type", "order", "payment", "popularity"]
result_df["year"] = year
result_df["month"] = month
result_df["day"] = day
result_df["place"] = place
result_df["race_id"] = race
result_all_df = result_all_df.append(result_df, ignore_index = True)
time.sleep(0.5)

# race info

soup = BeautifulSoup(response.content)
race_info = soup.findAll("diary_snap_cut")[0].text
race_info2 = soup.find("p", class_ = "smalltxt").text
race_info_df = race_info_df.append({"race_info":race_info, "race_info2":race_info2}, ignore_index = True)
time.sleep(0.5)

NameError: name 'year' is not defined

In [ ]:
url = "https://db.netkeiba.com/horse/ped/2019104730/"

In [ ]:
tabs = pd.read_html(url)

In [ ]:
tabs[0]

,0,1,2,3,4
0,イスラボニータ 2011 黒鹿毛 [血統][産駒] Halo系,フジキセキ 1992 青鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],Halo 1969 黒鹿毛,Hail to Reason
1,イスラボニータ 2011 黒鹿毛 [血統][産駒] Halo系,フジキセキ 1992 青鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],Halo 1969 黒鹿毛,Cosmah
2,イスラボニータ 2011 黒鹿毛 [血統][産駒] Halo系,フジキセキ 1992 青鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],Wishing Well 1975 鹿毛,Understanding
3,イスラボニータ 2011 黒鹿毛 [血統][産駒] Halo系,フジキセキ 1992 青鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],Wishing Well 1975 鹿毛,Mountain Flower
4,イスラボニータ 2011 黒鹿毛 [血統][産駒] Halo系,フジキセキ 1992 青鹿毛 [血統][産駒],ミルレーサー Millracer(米) 1983 鹿毛 [血統][産駒],Le Fabuleux 1961 鹿毛,Wild Risk
5,イスラボニータ 2011 黒鹿毛 [血統][産駒] Halo系,フジキセキ 1992 青鹿毛 [血統][産駒],ミルレーサー Millracer(米) 1983 鹿毛 [血統][産駒],Le Fabuleux 1961 鹿毛,Anguar
6,イスラボニータ 2011 黒鹿毛 [血統][産駒] Halo系,フジキセキ 1992 青鹿毛 [血統][産駒],ミルレーサー Millracer(米) 1983 鹿毛 [血統][産駒],Marston's Mill 1975 黒鹿毛,In Reality
7,イスラボニータ 2011 黒鹿毛 [血統][産駒] Halo系,フジキセキ 1992 青鹿毛 [血統][産駒],ミルレーサー Millracer(米) 1983 鹿毛 [血統][産駒],Marston's Mill 1975 黒鹿毛,Millicent
8,イスラボニータ 2011 黒鹿毛 [血統][産駒] Halo系,イスラコジーン Isla Cozzene(米) 2002 鹿毛 [血統][産駒],Cozzene 1980 芦毛 [血統][産駒],Caro 1967 芦毛,フォルティノ
9,イスラボニータ 2011 黒鹿毛 [血統][産駒] Halo系,イスラコジーン Isla Cozzene(米) 2002 鹿毛 [血統][産駒],Cozzene 1980 芦毛 [血統][産駒],Caro 1967 芦毛,Chambord
